In [1]:
from src.config import CACHE_DIR
from src.custom_embeddings import bge_m3_embed, qwen2_embed, nomic_embed
from src.vectorstorage import EmbeddingVectorStorage

print(f"Cache directory: {CACHE_DIR}")

Cache directory: /Users/arian/Documents/FHNW/npr/npr_hs_24/npr_mc1_new_hs24/npr-mc1-frfr/data/cache


In [2]:
# BAAI_bge_m3_RecursiveCharacterTextSplitter
# Alibaba-NLP_gte-Qwen2-7B_RecursiveCharacterTextSplitter
# nomic-ai_nomic-embed-text-v1_5_RecursiveCharacterTextSplitter

collection_name = "BAAI_bge_m3_RecursiveCharacterTextSplitter" 
path_persistent = "../data/chroma"

# Reload the vector store
bge_m3_vectordb = EmbeddingVectorStorage(
    method_of_embedding=bge_m3_embed,
    collection=collection_name,
    path_persistent=path_persistent
)

print(bge_m3_vectordb.test_heartbeat())  # Check if the Chroma client is active
print(bge_m3_vectordb.collection_is_empty())  # Check if the collection is empty

1732111263561074000
False


In [3]:
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM

qwen2_5_3B = 'qwen2.5:3b-instruct-q4_0'

basic_retriever = bge_m3_vectordb.as_retriever()
local_llm_model = OllamaLLM(model=qwen2_5_3B)

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from dotenv import load_dotenv

load_dotenv()

# https://python.langchain.com/v0.1/docs/use_cases/question_answering/sources/
# prompt: https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=2d6cd9b7-5b49-44db-a523-a13c23f12f29
basic_prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | basic_prompt
    | local_llm_model
    | StrOutputParser()
)

basic_rag_chain = RunnableParallel(
    {"context": basic_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

basic_rag_chain.invoke("The company is also aiming to reduce gas flaring?")

In [ ]:
from langchain_openai import ChatOpenAI
from src.evaluation import RAGEvaluation

# Initialize the class
openai_llm_model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
evaluator = RAGEvaluation(
    name="Baseline",
    rag_chain=basic_rag_chain,
    llm_model=openai_llm_model,
    embeddings=bge_m3_embed
)

clean_file_path='../data/preprocessed/clean_cleantech.parquet'
eval_file_path='../data/eval_dataset/cleantech_rag_evaluation_data_2024-09-20.csv'

# Perform evaluation
results = evaluator.evaluate(clean_file_path, eval_file_path, bge_m3_vectordb)

In [ ]:
evaluator.plot_results_all(results, results)

In [ ]:
evaluator.plot_eval_result(results)